In [13]:
# --- RETO 1 ---

# --- Importar ---
import os
import shutil
from pathlib import Path
from datetime import datetime

nombre_proyecto = "Proyecto_Scraping" 

# Obtener el directorio actual de trabajo (Current Working Directory)
base_dir = Path.cwd()
print(f"Estamos trabajando en: {base_dir}")

# Crear la ruta en memoria
carpeta_datos = base_dir / nombre_proyecto
print(f"\nRuta construida: {carpeta_datos}")

# Crear la ruta de las carpetas
carpetas_a_crear = [
    carpeta_datos / "data_raw",
    carpeta_datos / "data_clean",
    carpeta_datos / "img",
    carpeta_datos / "logs",
]

# Crear la función para crear las carpetas
def crear_carpetas(lista_carpetas):
    """
    Función auxiliar para crear las carpetas en la ruta específicada
    """
    # 1. Crear la carpeta física si no existe
    # parents=True crea carpetas intermedias si faltan
    # exist_ok=True evita errores si la carpeta ya existe
    for carpetas in lista_carpetas:
        carpetas.mkdir(parents=True, exist_ok=True)
        print(f"La carpeta {carpetas} fue creada con éxito")

crear_carpetas(carpetas_a_crear)

#  EXTRA: Crear el archivo readme.txt
def create_readme(logs_folder):
    """
    Función auxiliar para crear el archivo "readme.txt"
    """
    readme_path = logs_folder / "readme.txt"
    print(f"El archivo fue creado con éxito")

    # Solo lo creamos si NO existe (idempotencia)
    if not readme_path.exists():
        fecha = datetime.now().strftime("%Y-%m-%d")
        texto = f"Este proyecto fue inicializado el {fecha}\n"
        readme_path.write_text(texto)

create_readme(carpeta_datos / "logs")

Estamos trabajando en: c:\Users\LENOVO\Documents\MEGA CURSO PYTHON\CURSO WEB SCRAPING Y AUTOMATIZACION DE TAREAS

Ruta construida: c:\Users\LENOVO\Documents\MEGA CURSO PYTHON\CURSO WEB SCRAPING Y AUTOMATIZACION DE TAREAS\Proyecto_Scraping
La carpeta c:\Users\LENOVO\Documents\MEGA CURSO PYTHON\CURSO WEB SCRAPING Y AUTOMATIZACION DE TAREAS\Proyecto_Scraping\data_raw fue creada con éxito
La carpeta c:\Users\LENOVO\Documents\MEGA CURSO PYTHON\CURSO WEB SCRAPING Y AUTOMATIZACION DE TAREAS\Proyecto_Scraping\data_clean fue creada con éxito
La carpeta c:\Users\LENOVO\Documents\MEGA CURSO PYTHON\CURSO WEB SCRAPING Y AUTOMATIZACION DE TAREAS\Proyecto_Scraping\img fue creada con éxito
La carpeta c:\Users\LENOVO\Documents\MEGA CURSO PYTHON\CURSO WEB SCRAPING Y AUTOMATIZACION DE TAREAS\Proyecto_Scraping\logs fue creada con éxito
El archivo fue creado con éxito


In [15]:
# --- RETO 2 ---

import requests
from bs4 import BeautifulSoup
import re  # Importamos la librería de expresiones regulares

def analizar_categoria(url_categoria):
    # Usar un "User-Agent" para disfrazarnos de navegador.
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    # Realizar la petición GET
    response = requests.get(url_categoria, headers=headers)

    # Verificar el estado (Status Code)
    # 200 = Éxito, 404 = No encontrado, 403 = Prohibido, 500 = Error del servidor
    if response.status_code == 200:
        print("\n✅ Conexión exitosa con el servidor.")
        html_content = response.text
    else:
        print(f"❌ Error en la conexión: {response.status_code}")

    # Crear carpeta si no existe y guardar
    ruta_raw = carpeta_datos / "data_raw"
    ruta_raw.mkdir(exist_ok=True) # Por seguridad

    titulo_categoria = url_categoria.split('/')[-2] # EXTRA: obtener el nombre de la categoría desde la URL automáticamente
    nombre_archivo = f"raw_{titulo_categoria}.html"
    ruta_completa = ruta_raw / nombre_archivo

    # Guardamos el HTML tal cual (encoding='utf-8' es vital para tildes y símbolos)
    with open(ruta_completa, 'w', encoding='utf-8') as f:
        f.write(response.text)
    print(f"Archivo crudo guardado en: {ruta_completa}")

    # Crear el objeto "Sopa"
    soup = BeautifulSoup(html_content, 'html.parser')
    libros_containers = soup.find_all('article', class_='product_pod')

    datos_libros = []

    for libro in libros_containers:
        try:
            # --- Extracción del Título ---
            titulo = libro.h3.a['title']
            
            # --- Extracción del Precio (Método Robusto) ---
            precio_texto = libro.find('p', class_='price_color').text
            
            # Usamos Regex para buscar solo números y puntos
            # Patrón r"[\d\.]+" significa: "Encuentra dígitos (0-9) o puntos (.) uno o más veces"
            match = re.search(r"[\d\.]+", precio_texto)
            
            if match:
                # Si encuentra números, tomamos solo esa parte y convertimos
                precio_numerico = float(match.group())
            else:
                # Si por alguna razón no hay precio, ponemos 0.0 o NaN
                precio_numerico = 0.0
            
            # --- Extracción de Estrellas (Rating) ---
            clases_rating = libro.find('p', class_='star-rating')['class']
            rating = clases_rating[1] if len(clases_rating) > 1 else "Not Rated"
            
            # Guardamos en diccionario
            datos_libros.append({
                "titulo": titulo,
                "precio": precio_numerico,
                "rating": rating
            })
            
        except Exception as e:
            print(f"Error parseando un libro: {e}")

    # Verificamos los resultados
    print(f"\n--- Muestra de datos extraídos ({len(datos_libros)} total) ---")
    for l in datos_libros[:5]:
        print(l)
    return datos_libros

# URL objetivo
url_mystery = "http://books.toscrape.com/catalogue/category/books/mystery_3/index.html"
url_scifi = "http://books.toscrape.com/catalogue/category/books/science-fiction_16/index.html"

libros_mystery = analizar_categoria(url_mystery)
libros_scifi   = analizar_categoria(url_scifi)

print("\nResultados: ")
print(f"Mystery: {len(libros_mystery)} libros")
print(f"Science Fiction: {len(libros_scifi)} libros")


✅ Conexión exitosa con el servidor.
Archivo crudo guardado en: c:\Users\LENOVO\Documents\MEGA CURSO PYTHON\CURSO WEB SCRAPING Y AUTOMATIZACION DE TAREAS\Proyecto_Scraping\data_raw\raw_mystery_3.html

--- Muestra de datos extraídos (20 total) ---
{'titulo': 'Sharp Objects', 'precio': 47.82, 'rating': 'Four'}
{'titulo': 'In a Dark, Dark Wood', 'precio': 19.63, 'rating': 'One'}
{'titulo': 'The Past Never Ends', 'precio': 56.5, 'rating': 'Four'}
{'titulo': 'A Murder in Time', 'precio': 16.64, 'rating': 'One'}
{'titulo': 'The Murder of Roger Ackroyd (Hercule Poirot #4)', 'precio': 44.1, 'rating': 'Four'}

✅ Conexión exitosa con el servidor.
Archivo crudo guardado en: c:\Users\LENOVO\Documents\MEGA CURSO PYTHON\CURSO WEB SCRAPING Y AUTOMATIZACION DE TAREAS\Proyecto_Scraping\data_raw\raw_science-fiction_16.html

--- Muestra de datos extraídos (16 total) ---
{'titulo': 'Mesaerion: The Best Science Fiction Stories 1800-1849', 'precio': 37.59, 'rating': 'One'}
{'titulo': 'Join', 'precio': 35.67